# Chapter 3: 编码的注意机制

本笔记本中正在使用的软件包：

In [63]:
from importlib.metadata import version
print("torch version:",version("torch"))

torch version: 2.9.1+cu130


- 这一章涵盖了注意力机制，LLM的引擎：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="600px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="700px">

## 3.1 长序列建模的问题

- 本节中没有代码
- 由于源语和目的语语法结构的差异，逐字翻译文本是不可行的：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="500px">

- 在引入transformer模型之前，编码器-解码器RNNs通常用于机器翻译任务
- 在这种设置中，编码器使用隐藏状态（神经网络中的一种中间层）处理来自源语言的一系列令牌，以生成整个输入序列的浓缩表示：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 使用注意机制捕获数据依赖关系

- 本节中无代码
- 通过注意机制，网络的文本生成解码器部分能够选择性地访问所有输入令牌，这意味着某些输入令牌在生成特定输出令牌时比其他输入令牌更重要：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="600px">

- transformers 中的自关注是一种旨在通过使序列中的每个位置能够参与并确定同一序列中每个其他位置的相关性来增强输入表示的技术

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="400px">

## 3.3 利用自我注意力关注输入的不同部分

### 3.3.1 一种无需训练权重的简单自注意力机制

- 本节解释了一种非常简化的自注意力机制，它不包含任何可训练的权重。
- 这纯粹是为了说明目的，并非transformers中使用的注意力机制。
- 下一节，即 3.3.2 节，将扩展这个简单的注意力机制，以实现真正的自注意力机制。
- 假设我们有一个输入序列$x^{(1)}$ to $x^{(T)}$
  - 输入是一段文本（例如，“你的旅程始于一步”），它已经按照第 2 章所述转换为词嵌入。
  - 例如，$x^{(1)}$ 是一个 d 维向量，表示单词“你的”，依此类推。
- **目标：** 计算输入序列 $x^{(1)}$ 到 $x^{(T)}$ 中每个元素 $x^{(i)}$ 的上下文向量 $z^{(i)}$（其中 $z$ 和 $x$ 具有相同的维度）。
  - 上下文向量 $z^{(i)}$ 是输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权和。
  - 上下文向量是特定于某个输入的“上下文”
    - 我们不再用 $x^{(i)}$ 作为任意输入标记的占位符，而是考虑第二个输入 $x^{(2)}$。
    - 为了继续举一个具体的例子，我们不再使用占位符 $z^{(i)}$，而是考虑第二个输出上下文向量 $z^{(2)}$。
    - 第二个上下文向量 $z^{(2)}$ 是所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权和，权重以第二个输入元素 $x^{(2)}$ 为基准。
    - 注意力权重决定了在计算 $z^{(2)}$ 时，每个输入元素对加权和的贡献程度。
    - 简而言之，可以将 $z^{(2)}$ 视为 $x^{(2)}$ 的修改版本，它还包含了与当前任务相关的所有其他输入元素的信息。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="500px">
- （请注意，为了减少视觉混乱，图中数字已截断至小数点后一位；类似地，其他数字也可能包含截断值。）

- 按照惯例，未归一化的注意力权重被称为“注意力得分”，而归一化的注意力得分（总和为 1）被称为“注意力权重”。

- 以下代码将逐步执行上图所示的操作。

<br>

- **步骤 1：** 计算未归一化注意力分数 $\omega$

- 假设我们使用第二个输入标记作为查询，即 $q^{(2)} = x^{(2)}$，我们通过点积计算未归一化注意力分数：
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 上述公式中，$\omega$ 是希腊字母“omega”，用于表示未归一化注意力分数。
    - $\omega_{21}$ 中的下标“21”表示输入序列中的第二个元素是用作针对输入序列元素 1 的查询

- 假设我们有如下输入句子，它已经按照第 3 章所述嵌入到 3 维向量中（为了便于说明，这里我们使用了一个非常小的嵌入维度，以便它可以完整地显示在页面上而无需换行）：

In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- 本书遵循机器学习和深度学习的常用约定，将训练样本表示为行，特征值表示为列；对于上图所示的张量，每一行代表一个词，每一列代表一个嵌入维度。）
- 本节的主要目标是演示如何使用第二个输入序列 $x^{(2)}$ 作为查询来计算上下文向量 $z^{(2)}$
- 图示展示了该过程的初始步骤，即通过点积运算计算 $x^{(2)}$ 与所有其他输入元素之间的注意力得分 ω

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="500px">

- 我们以输入序列元素 $x^{(2)}$ 为例来计算上下文向量 $z^{(2)}$；在本节稍后部分，我们将推广此方法以计算所有上下文向量。
- 第一步是通过计算查询 $x^{(2)}$ 与所有其他输入标记之间的点积来计算未归一化的注意力分数：

In [2]:
query = inputs[1] #第二个输入标记是查询
attn_scores_2 = torch.empty(inputs.shape[0])
for i,x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i,query)# 点积（由于它们是一维向量，因此无需转置）
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 附注：点积本质上是将两个向量逐元素相乘，然后将结果相加的简写形式：

In [3]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **步骤 2：** 将未归一化的注意力得分（“omega”，$\omega$）归一化，使其总和为 1
- 以下是一种将未归一化的注意力得分归一化为总和为 1 的简单方法（这是一种约定俗成的做法，便于解释，并且对训练稳定性至关重要）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="600px">

In [4]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights 注意力权重:", attn_weights_2_tmp)
print("Sum 总和:", attn_weights_2_tmp.sum())

Attention weights 注意力权重: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum 总和: tensor(1.0000)


- 然而，在实践中，使用 softmax 函数进行归一化更为常见且推荐，因为它能更好地处理极端值，并且在训练过程中具有更理想的梯度特性。
- 以下是一个简单的 softmax 函数实现，用于缩放，同时对向量元素进行归一化，使其之和为 1：

In [5]:
def softmax_naive(x):
    return torch.exp(x)/ torch.exp(x).sum(dim =0)
attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights 注意力权重:", attn_weights_2_naive)
print("Sum 总和:", attn_weights_2_naive.sum())

Attention weights 注意力权重: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum 总和: tensor(1.)


- 上述简单实现对于过大或过小的输入值可能会出现数值不稳定问题，这是由于溢出和下溢造成的。
- 因此，在实践中，建议使用 PyTorch 实现的 softmax 函数，该函数针对性能进行了高度优化：

In [6]:
attn_weights_2 = torch.softmax(attn_scores_2,dim =0)

print("Attention weights 注意力权重:", attn_weights_2)
print("Sum 总和:", attn_weights_2.sum())

Attention weights 注意力权重: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum 总和: tensor(1.)


- **步骤 3**：通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘来计算上下文向量 $z^{(2)}$，并将结果向量相加：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="600px">

In [7]:
query = inputs[1]  # 第二个输入标记是查询
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)


tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有输入词元的注意力权重

#### 推广到所有输入序列标记：
- 上面，我们计算了输入 2 的注意力权重和上下文向量（如下图高亮行所示）。
- 接下来，我们将此计算推广到所有输入序列标记，以计算所有输入序列标记的注意力权重和上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="500px">

- （请注意，为了减少视觉混乱，图中数字已截断至小数点后两位；每行数值之和应为 1.0 或 100%；类似地，其他数字也已截断。）

- 在自注意力机制中，首先计算注意力得分，然后进行归一化处理，得到总和为 1 的注意力权重。
- 然后，利用这些注意力权重，通过对输入进行加权求和，生成上下文向量。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="500px">

- 将前面的**步骤 1**应用于所有成对元素，以计算未归一化的注意力得分矩阵：

In [8]:
attn_score = torch.empty(6,6)
for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
        attn_score[i,j]= torch.dot(x_i,x_j)
print(attn_score)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 我们可以通过矩阵乘法更高效地实现上述目标：

In [9]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与之前的步骤 2 类似，我们对每一行进行归一化，使每一行的值之和为 1：

In [10]:
attn_weights = torch.softmax(attn_score,dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每一行中的值之和是否确实为 1：

In [14]:
row_1_sum = sum([0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452])
print("Row 1 sum:",row_1_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 1 sum: 0.9999
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用前面的**步骤 3** 来计算所有上下文向量：

In [15]:
all_context_vecs = attn_weights @ inputs

print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 作为健全性检查，先前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上面的第 2 行中找到：

In [16]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 使用可训练权重实现自注意力机制

- 一个概念框架，阐明了本节中开发的自我注意机制如何融入本书和本章的整体叙述和结构中。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="500px">

### 3.4.1 逐级计算注意权值

- 在本节中，我们将实现在原始transformer架构、GPT模型和大多数其他流行的llm中使用的自关注机制
- 这种自我注意机制也被称为“缩放点积注意”。
- 整体思路与之前相似：
  - 我们想要计算上下文向量作为特定于某个输入元素的输入向量的加权和
  - 对于上述，我们需要注意权重
- 你会看到，与之前介绍的基本注意力机制相比，只有细微的区别：
  - 最显著的区别是引入了在模型训练期间更新的权重矩阵
  - 这些可训练的权重矩阵是至关重要的，这样模型（特别是模型中的注意力模块）就可以学会产生“好的”上下文向量

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="700px">

- 逐步实现self-attention机制，我们首先介绍三个训练权重矩阵$W_q$、$W_k$和$W_v$
- 这三个矩阵用于通过矩阵乘法将嵌入的输入标记 $x^{(i)}$ 投影到查询、键和值向量中：


  - 查询向量： $q^{(i)} = x^{(i)}\,W_q $
  - 关键向量： $k^{(i)} = x^{(i)}\,W_k $
  - 值向量：   $v^{(i)} =x^{(i)}\,W_v $


- 输入$x$和查询向量$q$的嵌入维度可以相同也可以不同，具体取决于模型的设计和具体实现
- 在GPT模型中，输入和输出维度通常是相同的，但为了便于说明，为了更好地遵循计算，我们在这里选择不同的输入和输出维度：

In [17]:
x_2 = inputs[2]        #第二个输入元素
d_in = inputs.shape[1] #输入嵌入大小，d=3
d_out = 2              #输出嵌入大小，d=2

- 下面，我们初始化三个权重矩阵；请注意，出于说明目的，我们设置`requires_grad=False`以减少输出中的混乱，
但如果我们要使用权重矩阵进行模型训练，我们将设置`requires_grad=True`以在模型训练期间更新这些矩阵

In [19]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 接下来我们计算查询、键和值向量：

In [25]:
query_2 = x_2 @ W_query # _2 因为它是相对于第二个输入元素
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4300, 1.4343])


- 如下所示，我们成功地将 6 个输入标记从 3D 投影到 2D 嵌入空间：

In [27]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步（**步骤 2**）中，我们通过计算查询和每个关键向量之间的点积来计算非标准化注意力分数：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="700px">

In [29]:
keys_2 = keys[1] # Python索引从0开始
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8284)


- 由于我们有 6 个输入，因此给定查询向量有 6 个注意力分数：

In [30]:
attn_scores_2 = query_2 @ keys.T ## 给定查询的所有注意力分数
print(attn_scores_2)

tensor([1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="700px">

- 接下来，在**步骤 3** 中，我们使用之前使用的 softmax 函数计算注意力权重（归一化注意力分数，总和为 1）
- 与之前的不同之处在于，我们现在通过将注意力分数除以嵌入维度的平方根 $\sqrt{d_k}$ （即 `d_k**0.5`）来缩放注意力分数：

In [31]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="700px">

- In **step 4**, we now compute the context vector for input query vector 2:

In [33]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3058, 0.8203])


### 3.4.2 实现紧凑的 SelfAttention 类

- 综上所述，我们可以实现如下的自注意力机制：

In [37]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))
    def forward(self,x):
        keys =x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega ω
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5,dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in,d_out)

print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="500px">

- 我们可以使用 PyTorch 的线性层简化上面的实现，如果我们禁用偏置单元，则相当于矩阵乘法
- 与我们的手动`nn.Parameter(torch.rand(...)`方法相比，使用`nn.Linear`的另一大优点是`nn.Linear`具有首选的权重初始化方案，这会导致更稳定的模型训练

In [40]:
class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self,x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 请注意，`SelfAttention_v1`和`SelfAttention_v2`给出不同的输出，因为它们对权重矩阵使用不同的初始权重

## 3.5 通过因果关注隐藏未来词

- 在因果注意力中，对角线上方的注意力权重被屏蔽，确保对于任何给定的输入，LLM 在使用注意力权重计算上下文向量时无法利用未来的标记

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="500px">

### 3.5.1 应用因果注意力掩模

- 在本节中，我们将之前的自注意力机制转换为因果自注意力机制
- 因果自注意力确保模型对序列中某个位置的预测仅依赖于先前位置的已知输出，而不依赖于未来位置
- 简单地说，这确保了每个下一个单词的预测应该只依赖于前面的单词
- 为了实现这一目标，对于每个给定的标记，我们屏蔽掉未来的标记（输入文本中当前标记之后的标记）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="700px">

- 为了说明和实现因果自注意力，让我们使用上一节中的注意力分数和权重：

In [44]:
# 重用查询和键权重矩阵
# 为了方便起见，上一节中的 SelfAttention_v2 对象

queries = sa_v2.W_query(inputs)
keys    = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim =-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 屏蔽未来注意力权重的最简单方法是通过 PyTorch 的 tril 函数创建一个掩码，其中主对角线下方的元素（包括对角线本身）设置为 1，主对角线上方的元素设置为 0：

In [45]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与该掩码相乘，将对角线上方的注意力分数归零：

In [46]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 但是，如果像上面那样在 softmax 之后应用 mask，则会破坏 softmax 创建的概率分布
- Softmax 确保所有输出值之和为 1
- softmax 之后的掩蔽需要将输出重新归一化，使其总和再次为 1，这会使过程复杂化，并可能导致意想不到的效果

- 为了确保行总和为 1，我们可以将注意力权重标准化如下：

In [49]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 虽然我们现在在技术上已经完成了因果注意机制的编码，但让我们简要地看一下实现与上述相同的更有效的方法
- 因此，我们可以在对角线上方的非标准化注意力分数进入 softmax 函数之前用负无穷大掩盖它们，而不是将对角线上方的注意力权重归零并重新标准化结果：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="550px">

In [50]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 正如我们在下面看到的，现在每行的注意力权重再次正确地总计为 1：

In [51]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 用 dropout 掩盖额外的注意力权重

- 此外，我们还应用dropout来减少训练过程中的过度拟合
-  Dropout可以应用在几个地方：
  - 例如，计算注意力权重之后；
  - 或者将注意力权重与值向量相乘之后
- 在这里，我们将在计算注意力权重后应用 dropout mask，因为它更常见
- 此外，在这个具体示例中，我们使用 50% 的退出率，这意味着随机屏蔽一半的注意力权重。
  （后面训练GPT模型的时候，我们会使用较低的dropout率，比如0.1或者0.2

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="5400px">

- 如果我们应用 0.5 (50%) 的丢弃率，则未丢弃的值将相应地缩放 1/0.5 = 2
- 缩放比例由公式 1 / (1 - `dropout_rate`) 计算

In [52]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # 丢失率为 50%
example = torch.ones(6, 6) #  创建一个由 1 组成的矩阵

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [53]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- 请注意，根据您的操作系统，最终的 dropout 输出可能看起来有所不同；您可以[在 PyTorch 问题跟踪器上](https://github.com/pytorch/pytorch/issues/121595) 阅读有关这种不一致的更多信息

### 3.5.3 实现紧凑的因果自注意力类

- 现在，我们准备好实施自我关注的有效实施，包括因果掩码和丢失掩码
- 还有一件事是实现代码来处理由多个输入组成的批次，以便我们的`CausalAttention`类支持我们在第 2 章中实现的数据加载器生成的批次输出
- 为简单起见，为了模拟此类批量输入，我们复制输入文本示例：

In [54]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) ## 2 个输入，每个输入有 6 个 token，每个 token 的嵌入维度为 3

torch.Size([2, 6, 3])


In [55]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # 新
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # 新批次尺寸 b
        # 对于“num_tokens”超过“context_length”的输入，这将导致错误
        # 在下面的掩码创建中。 
        # 实际上，这不是问题，因为LLM（第 4-7 章）确保输入
        # 在到达此转发方法之前不要超过`context_length`。
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # 改变转置transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place新的，_操作已就位
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` 用于解决批次中令牌数量小于支持的 context_size 的情况
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # 新

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，dropout 仅在训练期间应用，而不在推理期间应用

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="600px">

## 3.6 将单头注意力扩展到多头注意力

- 下面是之前实现的自注意力的总结（为简单起见，未显示因果掩码和丢失掩码）
- 这也称为单头注意力：

  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="500px">

- 我们简单地堆叠多个单头注意力模块以获得多头注意力模块:

  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="500px">

- 多头注意力背后的主要思想是使用不同的学习线性投影多次（并行）运行注意力机制。这使得模型能够共同关注来自不同位置的不同表示子空间的信息。

In [57]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上面的实现中，嵌入维度为 4，因为我们将`d_out=2`作为键、查询和值向量以及上下文向量的嵌入维度。由于我们有 2 个注意力头，因此我们的输出嵌入维度为 2*2=4

### 3.6.2 通过权重分割实现多头注意力

- 虽然上面是多头注意力的直观且功能齐全的实现（包装了之前的单头注意力`CausalAttention`实现），但我们可以编写一个名为 `MultiHeadAttention`的独立类来实现相同的效果
- 我们不会为这个独立的 `MultiHeadAttention`类连接单个注意力头
- 相反，我们创建单个 W_query、W_key 和 W_value 权重矩阵，然后将它们拆分为每个注意力头的单独矩阵：

In [58]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # # 减少投影暗度以匹配所需的输出暗度

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # 如 `CausalAttention` 中，对于 `num_tokens` 超过 `context_length` 的输入，
        # 这将导致下面的掩码创建出现错误。 
        # 实际上，这不是问题，因为LLM（第 4-7 章）确保输入
        # 在到达此转发方法之前不要超过`context_length`。
        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 我们通过添加“num_heads”维度隐式分割矩阵
        # 展开最后一个暗淡： (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # 转置： (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        # 使用因果掩模计算缩放点积注意力（也称为自注意力）
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        # 原始掩码被截断为tokens数量并转换为布尔值
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        # 使用掩码来填充注意力分数
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        # 形式：(b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        # 合并头，其中 self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，上面本质上是 `MultiHeadAttentionWrapper` 的重写版本，效率更高
- 由于随机权重初始化不同，结果输出看起来有点不同，但两者都是功能齐全的实现，可以在我们将在接下来的章节中实现的 GPT 类中使用

---

**关于输出尺寸的注释**

-  在上面`MultiHeadAttention` 中, 我使用了 `d_out=2` 来使用与之前`MultiHeadAttention` 类中相同的设置
- 由于串联，`MultiHeadAttention`返回输出头维度 `d_out * num_heads` (即 `2*2 = 4`)
-  然而，`MultiHeadAttention`类（使其更加用户友好）允许我们直接通过`d_out`控制输出头尺寸；这意味着，如果我们设置 `d_out = 2`，则无论头数有多少，输出头尺寸将为2
- 事后看来, 正如读者 [指出](https://github.com/rasbt/LLMs-from-scratch/pull/859), 使用`MultiHeadAttention`和`d_out = 4`可能更直观，这样它会产生与`MultiHeadAttentionWrapper`和`d_out = 2`相同的输出尺寸。

---

- 请注意，此外，我们在上面的`MultiHeadAttention`类中添加了一个线性投影层（`self.out_proj`）。这只是一个不改变维度的线性变换。在LLM实现中使用这样的投影层是一个标准约定，但这并不是绝对必要的（最近的研究表明，可以在不影响建模性能的情况下删除它；请参阅本章末尾的进一步阅读部分）

  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="500px">

- 请注意，如果您对上述内容的紧凑且高效的实现感兴趣，您还可以考虑 PyTorch 中的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)类

- 由于上面的实现乍一看可能有点复杂，让我们看看执行 `attn_scores = requests @keys.transpose(2, 3)` 时会发生什么：

In [59]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现将处理 4 维输入张量，以便在最后 2 个维度（num_tokens、head_dim）之间执行矩阵乘法，然后对各个头重复

- 例如，以下成为单独计算每个头的矩阵乘法的更紧凑的方法：

In [60]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 总结和要点

- 请参阅 [./multihead-attention.ipynb](./multihead-attention.ipynb) 代码笔记本，它是数据加载器（第 2 章）的简洁版本，加上我们在本章中实现的多头注意力类，在后续章节中训练 GPT 模型时将需要它
- 您可以在[./exercise-solutions.ipynb](./exercise-solutions.ipynb)中找到练习解决方案